# Informe LEC 4 - Estadística Computacional 2021-1 - Paralelo 01

## Integrantes:
* Renato Burgos - 201973021-5
* Bruno Liberona - 201973013-4
* Bastián Rámila - 201973003-7

## ***¡Se deben importar las siguientes librerías con tal de que las secciones de códigos funcionen!***

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as sp
import random
import statsmodels.api as sm
from scipy.special import gamma

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


## Ítem 1: _Distribución de la media muestral_

El tiempo que una persona demora en leer el libro "Don Quijote de la Mancha" es una variable aleatoria que sigue una distribución normal con promedio 21.72 horas y desviación estándar 2.3 horas.

#### ***a)*** Sea $X$ la media de una muestra aleatoria de tamaño $n - 20$ de tiempos de lecturas de este libro. ¿Cuál es en teoría la esperanza de $X$ y la desviación estándar de $X$?

#### ***b)*** Para comprobar los valores de a), obtenga $10^5$ muestras aleatorias de tamaño 20, luego obtenga la media de cada muestra. Calcule la media y la desviación estándar de la muestra de medias que obtuvo. ¿Se esperaba estos valores? Comente.

#### ***c)*** Repita el proceso de b), pero ahora para un tamaño de muestra aleatoria de $n - 40$ en vez de $n - 20$. Contraste en un solo gráfico la distribución de la muestra de medias obtenido en b) versus la muestra obtenida ahora. Comente lo observado.

#### ***d)*** ¿Cuál es la probabilidad de que el tiempo medio de lectura sea mayor a 24 horas cuando $n - 2$?

#### ***e)*** Presente conclusiones.

## Ítem 2: _Intervalos de confianza_

Se está estudiando la calidad del agua de grifo de cierta localidad. Para esto se tomaron muestras aleatorias del TDS del agua de las casas de esta zona. El TDS es el porcentaje de residuo seco que engloba sales inorgánicas y pequeñas cantidades de materia orgánica disueltas en el agua. Sus principales constituyentes son los cationes de calcio, magnesio, sodio y potasio; y los aniones de carbonato, bicarbonato, cloro, sulfato y nitrato. Este se mide en mg/l. Supongo que la distribución del TDS del agua para esta localidad es una normal con una desviación estándar de 30 mg/l.

#### ***a)*** Calcule un intervalo de confianza de 95% para el TDS promedio si el promedio en una muestra de tamaño 30 fue de 475 mg/l.

#### ***b)*** Calcule un intervalo de confianza de 90% para el TDS promedio si el promedio en una muestra de tamaño 20 fue de 475 mg/l.

#### ***c)*** ¿De qué tamaño debe ser una muestra si se quiere que el ancho del intervalo de 95% de confianza sea de 1 mg/l?

#### ***d)*** Como experimento, obtenga $10^5$ muestras aleatorias de tamaño 20 a partir de una distribución normal, con la desviación estándar mencionada previamente y una media poblacional $\mu - 500$. Para cada una de estas calcule el intervalo de confianza del 95% para su media muestral. ¿Qué porcentaje de estos intervalos inculye la media real de la población?

#### ***e)*** ¿Cree que cambiaría este porcentaje si aumenta el tamaño de las muestras? Argumente.

Ahora suponga que también se está estudiando el consumo de agua de las personas de sta localidad. Se desea calcular la proporción de hogares que consumen agua embotellada en vez de agua de la llave.

#### ***a)*** Luego de encuestar a 20 hogares se calculó que el porcentaje de consumo de agua embotellada es del 65%. Obtenga el intervalo del 95% de confianza para la proporción de la población.

#### ***b)*** ¿Qué método o fórmula utilizó para calcular los límites del IC? ¿Por qué el intervalo de confianza de la proporción no se calcula igual al calculado en a) y b)?

#### ***c)*** Presente conclusiones.

## Ítem 3: _Estimación de parámetros usando máxima verosimilitud_

La estimación de parámetros usando máxima verosimilitud se usa para determinar los parámetros de una distribución de probabilidad basándose en muestras. Suponga que se observan los valores $X_1, X_2, ..., X_n$ de $n$ variables aleatorias iid. El objetivo del método es estimar los parámetros de la distribución $(\theta)$ a partir de los valores observados. La función de verosimilitud $L(\theta) = P(X_1, ..., X_n|\theta)$ nos indica cuál es la probabilidad de que se observen esos valores para cierto parámetro $\theta$. Esta función se define como $L(\theta|X) = f(x_i, \theta)$. El método consiste en buscar el parámetro $\theta$ que maximice ésta función.

Usualmente se hace más sencillo trabajar con la función de log-verosimilitud $(\theta) = log L(\theta)$, debido a que el logaritmo es una función que preserva el orden.

Ahora para implementar este método, suponga que la muestra $X_1, X_2, ..., X_n$ proviene de una distribución exponencial con parámetro $\lambda$. Sabemos que la función de densidad de esta distribución viene dada por:



#### ***a)*** Genere una muestra de tamaño 1000 de esta distribución usando $\lambda = 2$.

#### ***b)*** Escriba una función que calcule la log-verosimilitud, tal que dependa de los datos observados y el parámetro $\lambda$. Grafique $(\lambda)$ en función de $\lambda$, dejando fijo el conjunto de datos los recién obtenidos.

#### ***c)*** Estime el valor de $\lambda$ que maximiza $(\lambda)$. Puede usar algún solver del módulo de scipy "scipy.optimize". Repita el gráfico anterior, pero ahora marque el punto óptimo encontrado.

A continuación repita el proceso anterior, pero ahora para estimar la desviación estándar de una distribución normal.

#### ***d)*** Genere una muestra de tamaño 1000 de una distribución normal usando $\mu = 15$ y  $\sigma = 1,5$.

#### ***e)*** Escriba una función que calcule la log-verosimilitud en base a los datos y $\sigma$.

#### ***e)*** Estime el valor de $\sigma$ que maximiza $(\sigma)$. Grafique $(\sigma)$ marcando el punto óptimo encontrado.